In [4]:
import pandas as pd
import xml.etree.ElementTree as ET
from tqdm import tqdm
import matplotlib.pyplot as plt
import os

In [5]:
def extract_elements_to_dict(element):
    data = {}
    for child in element:
        tag = child.tag
        if (tag!="relations" and tag in ["id","title","teaser","body","keywords","contentCreationDate","language"]):
            text = child.text.strip() if child.text else ''
            if len(child) > 0:
                def get_all_text(el):
                    texts = []
                    if el.text and el.text.strip():
                        texts.append(el.text.strip())
                    for c in el:
                        texts.append(get_all_text(c))
                    if el.tail and el.tail.strip():
                        texts.append(el.tail.strip())
                    return ' '.join(texts)
                
                text = get_all_text(child).strip()

            data[tag] = text
    return data

In [6]:
root_path = 'C:/Users/barsro/Downloads/Data_projets_EU/'
dirs = [name for name in os.listdir(root_path) if os.path.isdir(os.path.join(root_path, name))]

rows = []
for dir in dirs :
    print(dir)
    folder_path = os.path.join(root_path,dir)
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    for file in tqdm(files):
        try :
            tree = ET.parse(os.path.join(folder_path,file))
            root = tree.getroot()
            result = extract_elements_to_dict(root)
            result["Domain"]=dir
            rows.append(result)
        except :
            print(os.path.join(folder_path,file))

Digital_Economy


100%|██████████| 4659/4659 [00:13<00:00, 347.22it/s] 


Energy


100%|██████████| 3323/3323 [00:08<00:00, 394.04it/s] 


Environment


...

Space


100%|██████████| 960/960 [00:02<00:00, 341.68it/s]


Transport


100%|██████████| 1900/1900 [00:05<00:00, 371.35it/s]

C:/Users/barsro/Downloads/Data_projets_EU/Transport\xml.zip


In [7]:
df = pd.DataFrame(rows)
df['year'] = pd.to_datetime(df['contentCreationDate']).dt.year
df.head()

,language,id,title,teaser,body,keywords,contentCreationDate,Domain,year
0,en,182009-smart-grid-redirects-computing-power-to...,Smart grid redirects computing power to heat h...,EU-funded researchers have developed a decentr...,The EU-funded project can benefit end users by...,"EeHPC, high performance computing, HPC, Qarnot...",2016-05-05 15:53:11,Digital_Economy,2016
1,en,182988-spanish-sme-aims-for-top-spot-in-virtua...,Spanish SME aims for top spot in virtual fitti...,Online clothing retailers will soon be able to...,With eCommerce sales set to increase by 18.4 %...,"Smart textiles, safety shoes, biometric sensors",2016-07-12 17:56:55,Digital_Economy,2016
2,en,188522-robotic-solutions-to-give-dementia-pati...,Robotic solutions to give dementia patients be...,The EU-funded MARIO project is developing a co...,"In the absence of a cure, solutions such as MA...","MARIO, Kompaï-2 robot, dementia, DOMEO",2016-09-09 15:16:41,Digital_Economy,2016
3,en,188757-wisers-free-tools-will-help-large-and-s...,WISER’s free tools will help large and small e...,Companies and governments are bombarded by bil...,The vast majority of SMEs have to sacrifice li...,"WISER, cyber security, threat, trojan horse, S...",2016-10-28 13:13:54,Digital_Economy,2016
4,en,190722-pcp-and-ppi-a-public-boost-to-societal-...,PCP and PPI: a public boost to societal challe...,Rising public needs and interests require not ...,"In the EU, two mechanisms are increasingly use...","PCP, PPI, R&D, innovation, intelligent transpo...",2016-12-09 16:28:14,Digital_Economy,2016


In [8]:
df.to_csv('./cordis_v1.csv')